In [2]:
!pip install openai
!pip install langchain_openai

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="KEY",
)


completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a smart home assistant. Your task is generating Trigger-Action Rules for Home Automation"},
    {"role": "user", "content": "I want to turn off lights when I am not at home"}
  ]
)



print(completion.choices[0].message)

ChatCompletionMessage(content='Sure! Here\'s a Trigger-Action Rule (TAR) for that scenario:\n\n**Trigger**: Your smartphone disconnects from the home Wi-Fi network (indicating that you have left home).\n\n**Action**: Turn off all the lights in your home.\n\nThis can be configured using your home automation system or application. Here is a more detailed rule if you are using a typical smart home platform:\n\n1. **Trigger**: If the smartphone’s presence detection shows "away".\n2. **Action**: \n   - Turn off light 1.\n   - Turn off light 2.\n   - ...\n   - Turn off light n.\n\nAlternatively, you could use geofencing capabilities where the trigger involves your smartphone leaving a specified geographic area around your home instead of just leaving the Wi-Fi network.', role='assistant', function_call=None, tool_calls=None)


In [6]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from time import time

os.environ['OPENAI_API_KEY'] = "KEY"

import subprocess
from openai import OpenAI

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

# prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Your task is generating Trigger-Action Rules for Home Automation")
prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Create some Trigger-Action rules based on the devices in the SmartThings account")

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

chain = (
    prompt 
    | model 
    | output_parser
)

input_data = {
    "devices_output": devices_output
}

result = chain.invoke(input_data)

print(result)


Sure! Based on the devices in your SmartThings account, here are some Trigger-Action rules that you can set up to automate your smart home:

### Rule 1: Turn on Aeotec Outlet 1 when motion is detected
- **Trigger**: Motion detected by the Motion Sensor
- **Action**: Turn on Aeotec Outlet 1

**Rule setup**:
1. **Trigger**: Motion Sensor detects motion.
2. **Action**: Turn on Aeotec Outlet 1 (metering-switch).

### Rule 2: Turn off Centralite Outlet 1 if no motion is detected for 15 minutes
- **Trigger**: No motion detected by the Motion Sensor for 15 minutes
- **Action**: Turn off Centralite Outlet 1

**Rule setup**:
1. **Trigger**: No motion detected by the Motion Sensor for 15 minutes.
2. **Action**: Turn off Centralite Outlet 1 (switch-power-smartplug).

### Rule 3: Notify if the temperature drops below 60°F
- **Trigger**: Temperature measured by the Motion Sensor drops below 60°F
- **Action**: Send a notification

**Rule setup**:
1. **Trigger**: Temperature measurement by the Motion

In [15]:
import os
import subprocess
import csv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = "KEY"

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

# Create the prompt
prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Create a dataset with 100 linear temporal logic formulas for Trigger Action rules based on the {devices_output}")

# Initialize the model and output parser
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

# Create the chain
chain = (
    prompt 
    | model 
    | output_parser
)

input_data = {
    "devices_output": devices_output
}

# Get the result from the chain
result = chain.invoke(input_data)

# Split the result into individual LTL formulas if necessary (assumes result is a single string with formulas separated by newlines)
ltl_formulas = result.split('\n')

# Write the LTL formulas to a CSV file
csv_file_path = 'ltl_formulas.csv'
with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['LTL Formula'])  # Write header
    for formula in ltl_formulas:
        csvwriter.writerow([formula])

print(f"LTL formulas have been written to {csv_file_path}")


LTL formulas have been written to ltl_formulas.csv


In [2]:
import os
import subprocess
import csv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = "KEY"

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

def generate_trigger_action_rule(devices_output):
    prompt = ChatPromptTemplate.from_template(
        "You are a smart home assistant. The devices are in {devices_output}. Create a Trigger-Action rule based on the devices in the SmartThings account"
    )
    output_parser = StrOutputParser()
    model = ChatOpenAI(model="gpt-4o")

    chain = (
        prompt 
        | model 
        | output_parser
    )

    input_data = {
        "devices_output": devices_output
    }

    result = chain.invoke(input_data)
    return result.strip()  # Assuming the result is a single string for one rule

def generate_ltl_formulas(devices_output):
    prompt = ChatPromptTemplate.from_template(
        "You are a smart home assistant. The devices are in {devices_output}. Create a dataset with 100 linear temporal logic formulas for Trigger Action rules based on the {devices_output}"
    )
    output_parser = StrOutputParser()
    model = ChatOpenAI(model="gpt-4o")

    chain = (
        prompt 
        | model 
        | output_parser
    )

    input_data = {
        "devices_output": devices_output
    }

    result = chain.invoke(input_data)
    return result.split('\n')  # Assuming the result is a single string with formulas separated by newlines

def check_trigger_action_rule_with_ltl(trigger_action_rule, ltl_formulas):
    """
    Checks if the Trigger-Action rule matches any of the LTL formulas using LLM.

    :param trigger_action_rule: Trigger-Action rule
    :param ltl_formulas: List of LTL formulas
    :return: True if the rule matches any LTL formula, False otherwise
    """
    prompt = ChatPromptTemplate.from_template(
        "You are a smart home assistant. The following is a Trigger-Action rule and its corresponding LTL formulas. Check if the rule complies with any of the LTL formulas. Respond with 'True' if the rule complies with any formula, otherwise respond with 'False'.\n\nTrigger-Action Rule:\n{trigger_action_rule}\n\nLTL Formulas:\n{ltl_formulas}"
    )
    output_parser = StrOutputParser()
    model = ChatOpenAI(model="gpt-4o")

    chain = (
        prompt 
        | model 
        | output_parser
    )

    input_data = {
        "trigger_action_rule": trigger_action_rule,
        "ltl_formulas": "\n".join(ltl_formulas)
    }

    result = chain.invoke(input_data)
    return result.strip().lower() == 'true'

def save_to_csv(data, file_path, header):
    with open(file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(header)  # Write header
        for item in data:
            csvwriter.writerow([item])
    print(f"Data has been written to {file_path}")

# Main loop
devices_output = list_smartthings_devices()
matched = False
iteration = 0

# Generate LTL formulas once
ltl_formulas = generate_ltl_formulas(devices_output)
print(f"Generated LTL Formulas:\n{ltl_formulas}")

while not matched and iteration < 10:  # Limit the iterations to avoid an infinite loop
    print(f"Iteration: {iteration+1}")
    trigger_action_rule = generate_trigger_action_rule(devices_output)
    print(f"Generated Trigger-Action Rule:\n{trigger_action_rule}")

    matched = check_trigger_action_rule_with_ltl(trigger_action_rule, ltl_formulas)

    if matched:
        print("Trigger-Action rule matches the LTL formulas.")
        save_to_csv([trigger_action_rule], 'trigger_action_rule.csv', ['Trigger Action Rule'])
    else:
        print("Trigger-Action rule does not match the LTL formulas. Generating new rule...")
    
    iteration += 1

# Save the LTL formulas to a CSV file
save_to_csv(ltl_formulas, 'ltl_formulas.csv', ['LTL Formula'])


Generated LTL Formulas:
['Sure, here\'s a dataset of 100 linear temporal logic (LTL) formulas for Trigger-Action rules based on the provided devices. Each formula follows the structure of "IF [trigger event] THEN [action]."', '', '### Trigger-Action Rules in LTL', '', '1. **G (motionSensor_main_motion -> F switch_main_switch_on)**', '    - If motion is detected by the Motion Sensor, the Aeotec Outlet 1 switch should turn on eventually.', '', '2. **G (switch_main_powerMeter_greater_than_100 -> F switch_main_switch_off)**', '    - If the power meter of Aeotec Outlet 1 reads more than 100W, the switch should turn off eventually.', '', '3. **G (temperatureMeasurement_main_temperature_above_30 -> F switch_main_switch_on)**', '    - If the temperature measured by the Motion Sensor is above 30°C, Aeotec Outlet 1 switch should turn on eventually.', '', '4. **G (motionSensor_main_motion -> F switch_power_smartplug_main_switch_on)**', '    - If motion is detected by the Motion Sensor, the Centra

In [ ]:
# Harper's code
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from time import time

os.environ['OPENAI_API_KEY'] = "KEY"

import subprocess
from openai import OpenAI

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

# prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Your task is generating Trigger-Action Rules for Home Automation")
prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Create some Trigger-Action rules based on the devices in the SmartThings account")

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

chain = (
    prompt 
    | model 
    | output_parser
)

input_data = {
    "devices_output": devices_output
}

result = chain.invoke(input_data)

print(result)